Задачи:

1) Выберите группы ВК для
исследования.

2) Выгрузите возраст подписчиков (подробнее см. документацию vk.com/dev/groups.getMembers).

3) Сравните средний возраст подписчиков.

4) Очистите данные от выбросов и заново сравните средний возраст.

In [12]:
import requests
import math
from pprint import pprint
import datetime
import pandas as pd
import numpy as np

In [7]:
access_token = 'ff657386ff657386ff657386f0ff0b1b2afff65ff657386a28c242ff8cbed4f0f5d7ee5'
api_version = '5.89'

In [8]:
parametrs = {
  'access_token': access_token,
  'v': api_version,
  'group_id': 'megapteka'
  }

method_name = 'groups.getById'
r = requests.get('https://api.vk.com/method/' + method_name, params=parametrs)
r.json()

{'response': [{'id': 157033364,
   'is_closed': 0,
   'name': 'MEGAPTEKA.RU⠀|⠀МЕГАПТЕКА.РУ — поиск лекарств',
   'photo_100': 'https://sun9-49.userapi.com/impf/c840025/v840025390/765bf/Fc9HvD-kw8Y.jpg?size=100x0&quality=96&crop=0,0,200,200&sign=36c7560f0672a816302fb6b3678bf795&c_uniq_tag=zG_VKpYJlufRaT3IH9TvPMkiCh0K-EwHN4dFLA5STKM&ava=1',
   'photo_200': 'https://sun9-49.userapi.com/impf/c840025/v840025390/765bf/Fc9HvD-kw8Y.jpg?size=200x0&quality=96&crop=0,0,200,200&sign=6c7418a8f86b9fa604714f59a35ff654&c_uniq_tag=Le8S5QhxcCgS-WTQaNLQoMJzJqWR4D1Id26XgUOS9QQ&ava=1',
   'photo_50': 'https://sun9-49.userapi.com/impf/c840025/v840025390/765bf/Fc9HvD-kw8Y.jpg?size=50x0&quality=96&crop=0,0,200,200&sign=83f512b8e618766868da6e63f88c8741&c_uniq_tag=1D8En4jekdJUyJ3h6osp_5DZNrcmrAyj0ckxw5y90Ac&ava=1',
   'screen_name': 'megapteka',
   'type': 'page'}]}

In [17]:
groups = ['domoidostavim', 'megapteka', 'eshfresh59', 'semyaperm', 'pizzburg']
now = datetime.datetime.now()
ages_of_groups = {}

Загружаем информацию пользователей группы VK, вычисляем возраст и добавляем в список

In [18]:
for group in groups:

  years = []

  parametrs = {
        'access_token': access_token,
        'v': api_version,
        'group_id': group,
        'fields': 'bdate',
        'offset': 0
  }
  method_name = 'groups.getMembers'
  r = requests.get('https://api.vk.com/method/' + method_name, params=parametrs)
  users = r.json()

  count = math.ceil(users['response']['count']/1000)

  for n in range(count):
       
    for user in users['response']['items']:
      if 'bdate' in user:
        x = user['bdate'].split('.')
        if len(x) == 3:
          years.append(now.year - int(x[2]))

    parametrs['offset'] += 1000
    r = requests.get('https://api.vk.com/method/' + method_name, params=parametrs)
    users = r.json()

  ages_of_groups[group] = years

In [21]:
ages_of_groups.keys()

dict_keys(['domoidostavim', 'megapteka', 'eshfresh59', 'semyaperm', 'pizzburg'])

In [22]:
for key, ages in ages_of_groups.items():
    print(key)
    print('Средний возраст группы', sum(ages) / len(ages))
    Q1, Q2, Q3 = np.percentile(ages, [25,50,75])
    IQR = Q3 - Q1
    upper_inner = Q3 + 1.5 * IQR
    lower_inner = Q1 - 1.5 * IQR
    print('lower_inner, upper_inner')
    print(lower_inner, upper_inner)
    print('Q1, Q3')
    print(Q1, Q3)
    clean_ages = [age
                  for age in ages
                  if lower_inner <= age <= upper_inner]
    print('Средний возраст группы ,без выбросов', sum(clean_ages) / len(clean_ages))
    print()

domoidostavim
Средний возраст группы 34.54127694184628
lower_inner, upper_inner
15.5 51.5
Q1, Q3
29.0 38.0
Средний возраст группы ,без выбросов 33.05905172413793

megapteka
Средний возраст группы 40.51127819548872
lower_inner, upper_inner
4.0 76.0
Q1, Q3
31.0 49.0
Средний возраст группы ,без выбросов 40.01300108342362

eshfresh59
Средний возраст группы 35.94444444444444
lower_inner, upper_inner
16.5 52.5
Q1, Q3
30.0 39.0
Средний возраст группы ,без выбросов 34.488146551724135

semyaperm
Средний возраст группы 34.61293168641291
lower_inner, upper_inner
11.5 55.5
Q1, Q3
28.0 39.0
Средний возраст группы ,без выбросов 33.118815789473686

pizzburg
Средний возраст группы 29.224124097352234
lower_inner, upper_inner
12.0 44.0
Q1, Q3
24.0 32.0
Средний возраст группы ,без выбросов 27.969352378310912

